In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

import os
import cv2
import glob
from scipy.stats import mode

## 우선 환자별로 ct사진의 ROI를 왼쪽부분과 오른쪽부분으로 나눈 데이터셋 필요 
## (x_left, y_left, x_right, y_right)--> 만들어보기

In [ ]:
for n in range(1, 2):
    
    random.seed(n)
    l = [i for i in range(1, 51)]
    random.shuffle(l)
    
    x_train_left = np.zeros((5000, 64, 64, 2))
    y_train_left = np.zeros((5000))
    
    x_train_right = np.zeros((5000, 64, 64, 2))
    y_train_right = np.zeros((5000))

    left_start = 0
    left_end = 0
    
    right_start = 0
    right_end = 0

    for j in l[:40]:
        if j < 10:
            j = '0' + str(j) + '/'
        else:
            j = str(j) + '/'

        # 이 부분은 ct 이미지에서 왼쪽부분에 대한 ROI를 추출하여 64x64로 resize한 데이터를 불러옴 (x_left.npy)
        x = np.load('/content/drive/Shareddrives/high_classification/train/%sx_left.npy' % (j))
        y = np.load('/content/drive/Shareddrives/high_classification/train/%sy_left.npy' % (j))

        y = y.astype(np.uint8)
        
        x_left = np.zeros((200, 64, 64, 2))
        y_left = np.zeros((200, ))
        c = 0
        for i in range(1, y.shape[0]):
            x1 = np.zeros((64, 64, 2))
            if y[i-1] == y[i]:
                x1[:, :, 0] = x[i-1]
                x1[:, :, 1] = x[i]

                x_left[i-1] = x1
                y_left[i-1] = y[i-1]
                c += 1
        x_left = x_left[:c]
        y_left = y_left[:c]
        
        if left_end == 0:
            left_end = x_left.shape[0]
        else:
            left_end += x_left.shape[0]
            
        x_train_left[left_start:left_end] = x_left
        y_train_left[left_start:left_end] = y_left
        
        left_start += x_left.shape[0]

        # ct 이미지에서 오른쪽부분에 대한 ROI를 추출하여 64x64로 resize한 데이터를 불러옴, 데이터 위치에 맞게 코드 변경

        x = np.load('/content/drive/Shareddrives/high_classification/train/%sx_right.npy' % (j))
        y = np.load('/content/drive/Shareddrives/high_classification/train/%sy_right.npy' % (j))

        y = y.astype(np.uint8)

        x_right = np.zeros((200, 64, 64, 2))
        y_right = np.zeros((200, ))
        
        c = 0
        for i in range(1, y.shape[0]):
            x1 = np.zeros((64, 64, 2))
            if y[i-1] == y[i]:
                x1[:, :, 0] = x[i-1]
                x1[:, :, 1] = x[i]

                x_right[i-1] = x1
                y_right[i-1] = y[i-1]
                c += 1
        x_right = x_right[:c]
        y_right = y_right[:c]
    
        if right_end == 0:
            right_end = x_right.shape[0]
        else:
            right_end += x_right.shape[0]

        x_train_right[right_start:right_end] = x_right
        y_train_right[right_start:right_end] = y_right

        right_start += x_right.shape[0]
        
    x_train_left = x_train_left[:left_start]  # x_train_left = x_train_left[:left_end]
    y_train_left = y_train_left[:left_start]
            
    x_train_right = x_train_right[:right_start]
    y_train_right = y_train_right[:right_start]

    x_train = np.vstack((x_train_left, x_train_right))
    y_train = np.hstack((y_train_left, y_train_right))
    
    # 데이터 저장
    np.save('/content/drive/Shareddrives/high_classification/data/2c/x_train_nearest.npy', x_train)
    np.save('/content/drive/Shareddrives/high_classification/data/2c/y_train_nearest.npy', y_train)

    print(x_train.shape)
    print(y_train.shape)